<a href="https://colab.research.google.com/github/PratikhyaManas/EnergyForecasting/blob/main/Forecasting_LSTMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import glob
import pandas as pd
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
os.chdir("/content/drive/My Drive/Dataset/")

In [5]:
extension = 'csv'
all_files = [i for i in glob.glob('*.{}'.format(extension))]

In [ ]:
#combine all files in the list
df = pd.concat([pd.read_csv(f, low_memory=False) for f in all_files])

In [ ]:
#export to csv
df.to_csv("appended.csv", index=False, encoding='utf-8-sig')

df['Date Time'] = pd.to_datetime(df['SETTLEMENTDATE'])
df['Day'] = df['Date Time'].dt.day
df['Month'] = df['Date Time'].dt.month
df['Year'] = df['Date Time'].dt.year
df['Hour'] = df['Date Time'].dt.hour
df['Minute'] = df['Date Time'].dt.minute

In [ ]:
df['Demand'] = pd.to_numeric(df['TOTALDEMAND'],errors='coerce')
df.drop(['REGION'],axis=1,inplace=True)
df.drop(['SETTLEMENTDATE'],axis=1,inplace=True)
df.drop(['TOTALDEMAND'],axis=1,inplace=True)
df.drop(['RRP'],axis=1,inplace=True)
df.drop(['PERIODTYPE'],axis=1,inplace=True)
df.drop(['Date Time'],axis=1,inplace=True)


In [ ]:
X = []
y = []
for i in range(0,df.shape[0]-48):
    X.append(df.iloc[i:i+48, 5])
    y.append(df.iloc[i+48, 5])

X, y = np.array(X), np.array(y)
y = np.reshape(y,(len(y), 1))

X = np.delete(X,list(range(1,X.shape[1],2)),axis=1)
X = np.delete(X,list(range(1,X.shape[0],2)),axis=0)
y = np.delete(y,list(range(1,y.shape[0],2)),axis=0)

pd.DataFrame(X).to_csv('appended_demand.csv')
pd.DataFrame(y).to_csv('appended_demand_1.csv')

In [6]:
X = pd.read_csv('appended_demand.csv')

In [7]:
y = pd.read_csv('appended_demand_1.csv')

In [8]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
X = scaler.fit_transform(X)
y = scaler.fit_transform(y)


X_train,X_test = X[:-480],X[-480:]
y_train,y_test = y[:-480],y[-480:]

X_train = np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))
X_test = np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))

In [ ]:
from tensorflow import keras
from tensorflow.keras import Sequential
# from tensorflow.python.keras.layers import LSTM, CuDNNLSTM

from keras import optimizers


model = Sequential()
model.add(keras.Input(shape=(X_train.shape[1],1)))
model.add(keras.layers.LSTM(50,return_sequences=True))
model.add(keras.layers.LSTM(50,return_sequences=False))
model.add(keras.layers.Dense(50,activation='relu'))
model.add(keras.layers.Dense(1))

from keras.callbacks import ModelCheckpoint, EarlyStopping
filepath = 'models/{epoch:02d}-{loss:.4f}-{val_loss:.4f}.hdf5'
callbacks = [EarlyStopping(monitor='val_loss',patience=50),
             ModelCheckpoint(filepath,monitor='loss',save_best_only=True,mode='min')]

opt = optimizers.Adam(lr=0.0001)
model.compile(optimizer=opt,
              loss='mean_squared_error',
              metrics=['accuracy'])
model.fit(X_train,y_train,validation_split=0.2,epochs=20,callbacks=callbacks,batch_size=256)

In [ ]:
model.load_weights('models/abc.hdf5')

import time
for i in range(0,X_test.shape[0]):
    demand_summary = []
    X_input = X_test[i, :, :]
    X_input = np.reshape(X_input,(1,X_input.shape[0],1))
    X_input = model.predict(X_input)
    forecast = scaler.inverse_transform(X_input)

    y_input = y_test[i,:]
    y_input = np.reshape(y_input,(1,1))
    actual = scaler.inverse_transform(y_input)

    demand_summary.append(actual)
    demand_summary.extend(forecast)

    df_animate = pd.DataFrame(demand_summary)
    df_animate = df_animate.T
    df_animate.to_csv('real_time_demand.csv',mode='a',header=False,index=False)

    print(demand_summary)
    time.sleep(0.5)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import style

style.use('fivethirtyeight')
fig = plt.figure()
fig.figsize = (8,4)
ax1 = fig.add_subplot(1,1,1)

def animate(i):
    df = pd.read_csv('real_time_demand.csv')
    ys_actual = df.iloc[:,0].values
    ys_forecast = df.iloc[:,1].values

    if len(ys_actual)>=120:
        ys_actual = df.iloc[-120:,0].values
        ys_forecast = df.iloc[-120:,1].values

    xs = list(range(1,len(ys_actual)+1))
    ax1.clear()
    ax1.plot(xs,ys_actual)
    ax1.plot(xs,ys_forecast)

    ax1.set_title('One Hour Ahead of Load Forecasting - Victoria,Australia(MW)')
    ax1.legend(['Actual','Forecast'],loc = 'lower_right')


ani = animation.FuncAnimation(fig,animate,interval=500)
plt.tight_layout()
plt.show()